In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# plt.style.use('_mpl-gallery')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
path = 'data/'

data = pd.read_excel(path+'case_study.xlsx', sheet='data')
sub = pd.read_excel(path+'case_study.xlsx', sheet='çıktı')

In [3]:
data.info()

In [4]:
sub.info()

In [5]:
def onehot_encode(df, column): # Tarih verilerinde kullanırım diye koydum, ama kullanmadım.
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [6]:
def preprocess_inputs(df1, df2):
    # Data ve Çıktı DataFrame'leri üzerinde tek seferde işlem yapabilmek için
    df = pd.concat([df1, df2], ignore_index=True)
    
    df['TARIH'] = pd.to_datetime(df['TARIH'])

    for column in ['Day', 'Month', 'Year']: # Tarih datasını gün, ay, ve yıl columnlarına parçalıyor
        df[column] = df['TARIH'].apply(lambda x: getattr(x, column.lower())).astype(int)
    
    df.drop(['TARIH', 'URUN_ID', 'URUN_GRUBU', 'TALEP TAHMIN'], axis=1, inplace=True) # Gereksiz sütunlar

    df['SATIS_ADET'] = df['SATIS_ADET'].fillna(0) # Az olduğundan ve bilinmediğinden
    df['STOK_DISI_SURE'] = df['STOK_DISI_SURE'].fillna(0) # NaN verileri 0 a eşitledim
    
    # Eksik olan tarihleri ekleyip,
    # SATIS_ADET değerini bir önceki gün ile sonraki gününkinin ortalamasına eşitliyor
    for year in [2017, 2018, 2019]:
        for month in range(1,13):
            for day in range(1, 32):
                if ((month == 2) & (day in [29, 30, 31])) | ((day == 31) & (month in [4, 6, 9, 11])):
                    continue
                elif df[(df.Day==day) & (df.Month==month) & (df.Year==year)].empty:
                    avg_satis_adet = (df.loc[index, 'SATIS_ADET'] + df.loc[index+1, 'SATIS_ADET'])/2
                    df.loc[index+.5] = avg_satis_adet, 0, day, month, year
                    df = df.sort_index().reset_index(drop=True)
                index = df[(df.Day==day) & (df.Month==month) & (df.Year==year)].index.tolist()[0]

    return df

In [7]:
df = preprocess_inputs(data, sub)

In [8]:
df.info()

In [9]:
df.describe()

In [10]:
plt.figure(figsize=(16, 12))
for i in range(len(df.columns)):
    plt.subplot(2, 3, i+1)
    sns.histplot(df[df.columns[i]], kde=True)
    if i == 0:
        plt.title("Column Distributions")
plt.show()

In [11]:
plt.figure(figsize=(16, 12))
for i in range(len(df.columns)):
    plt.subplot(2, 3, i+1)
    sns.boxplot(df[df.columns[i]])
plt.show()

In [12]:
df_test = df.copy()

In [13]:
max_SATIS_ADET = df_test.SATIS_ADET.quantile(.75)*1.5 # Outliers lardan kurtulmak için
df_test['SATIS_ADET'][df_test['SATIS_ADET'] > max_SATIS_ADET] = max_SATIS_ADET

In [14]:
df_avg1 = (df_test
          .loc[:1063, ('Day', 'STOK_DISI_SURE')]
          .groupby(by=['Day'], as_index=False)
          .mean()
          .rename(columns={'STOK_DISI_SURE': 'AVG_STOK_DISI_SURE_DAY'})
         ) # Güne göre gruplayıp ortalama stok dışı süreyi buluyor
df_avg2 = (df_test
          .loc[:1063, ('Month', 'STOK_DISI_SURE')]
          .groupby(by=['Month'], as_index=False)
          .mean()
          .rename(columns={'STOK_DISI_SURE': 'AVG_STOK_DISI_SURE_MONTH'})
         ) # Aya göre gruplayıp ortalama stok dışı süreyi buluyor
df_avg3 = (df_test
          .loc[:1063, ('Day', 'Month', 'SATIS_ADET')]
          .groupby(by=['Month', 'Day'], as_index=False)
          .mean()
          .rename(columns={'SATIS_ADET': 'AVG_SATIS'})
         ) # Güne ve Aya göre gruplayıp ortalama satış buluyor

In [15]:
df_avg1

In [16]:
df_test = pd.merge(df_test, df_avg1, how='left', on=['Day'])

In [17]:
df_test = pd.merge(df_test, df_avg2, how='left', on=['Month'])

In [18]:
df_test = pd.merge(df_test, df_avg3, how='left', on=['Month', 'Day'])

In [19]:
df_test # Önceden oluşturduğumuz DataFrame'lerin birleşmiş hali

In [20]:
# Eğer stok dışı süresi 0 dan farklı ise ortalama stok dışı süresi ile ortalama satışı çarpıyoruz,
# Buradaki düşüncem ne kadar uzun süre stok dışında kaldı ise bu satır bizim için o kadar değerli
df_test['SATIS_STOK_DAY'] = pd.DataFrame(np.where(df_test.AVG_STOK_DISI_SURE_DAY > 0, df_test.AVG_SATIS * df_test.AVG_STOK_DISI_SURE_DAY, df_test.AVG_SATIS))
df_test['SATIS_STOK_MONTH'] = pd.DataFrame(np.where(df_test.AVG_STOK_DISI_SURE_MONTH > 0, df_test.AVG_SATIS * df_test.AVG_STOK_DISI_SURE_MONTH, df_test.AVG_SATIS))

In [21]:
plt.figure(figsize=(16, 16))
sns.pairplot(df_test.loc[:, ['SATIS_ADET', 'AVG_STOK_DISI_SURE_DAY', 'AVG_STOK_DISI_SURE_MONTH', 'AVG_SATIS', 'SATIS_STOK_DAY', 'SATIS_STOK_MONTH']].sample(1000), plot_kws=dict(alpha=.6, edgecolor='none'))
plt.show()

In [22]:
plt.figure(figsize=(10,10))
sns.heatmap(df_test.loc[:, ['SATIS_ADET', 'AVG_STOK_DISI_SURE_DAY', 'AVG_STOK_DISI_SURE_MONTH', 'AVG_SATIS', 'SATIS_STOK_DAY', 'SATIS_STOK_MONTH']].corr(), annot=True, vmin=-1.0,cmap="YlGnBu")
plt.show()

In [23]:
def scale_split_validate(df): # Bir çok modeli hızlıca deneyip test etmek için
    
    # Feature Engineering
    
#     for column in ['Day', 'Month', 'Year']:
#         df = onehot_encode(df, column=column)
    # ------------------
    
    df = df.copy().iloc[:-31]

    y = df['SATIS_ADET']
    X = df.drop(['SATIS_ADET', 'STOK_DISI_SURE'], axis=1)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
    
    models = {
        "    Linear Regression": LinearRegression(),
        "(L2) Ridge Regression": Ridge(),
        "(L1) Lasso Regression": Lasso(),
        "  K-Nearest Neighbors": KNeighborsRegressor(),
        "        Decision Tree": DecisionTreeRegressor(),
        "        Random Forest": RandomForestRegressor(),
        "    Gradient Boosting": GradientBoostingRegressor()
    }

    for name, model in models.items():
        model.fit(X_train, y_train)
        print(name + " trained.")
    print('*'*10)

    for name, model in models.items():
        r2 = model.score(X_test, y_test)
        print(name + " Test R^2 Score: {:.4f}".format(r2))
    print('---'*5)

#     for name, model in models.items():
#         print(name + "  Val R^2 Score: {:.4f}".format(model.score(X_val, y_val)))
#     print('---'*5)

    for name, model in models.items():
        y_pred = model.predict(X_test)
        rmse = np.sqrt(np.mean((y_test - y_pred)**2))
        print(name + " RMSE: {:.2f}".format(rmse))

In [24]:
scale_split_validate(df_test)

In [25]:
df_test.iloc[-31:]

In [26]:
df_pred = df_test.copy()
df_pred

In [27]:
X_pred = df_pred.iloc[-31:].drop(['SATIS_ADET', 'STOK_DISI_SURE'], axis=1)
df_pred = df_pred.iloc[:-31]

In [28]:
X_pred

In [29]:
df_pred

In [30]:
y_train = df_pred['SATIS_ADET']
X_train = df_pred.drop(['SATIS_ADET', 'STOK_DISI_SURE'], axis=1)

In [31]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

name, model = "(L1) Lasso Regression", Lasso()

model.fit(X_train, y_train)
print(name + " trained.")
print('---'*5)

r2 = model.score(X_train, y_train)
print(name + " Test R^2 Score: {:.4f}".format(r2))
print('---'*5)

In [32]:
X_pred = scaler.fit_transform(X_pred)

In [33]:
y_pred = model.predict(X_pred)

In [34]:
y_pred = pd.DataFrame(y_pred).rename(columns={0: 'TALEP TAHMIN'})

In [35]:
sub['TALEP TAHMIN'] = y_pred

In [36]:
sub

In [37]:
path = './'
sub.to_csv(path + "submission.csv")